# Broadcasting

Broadcasting is a mechanism that allows tensors of different shapes to be used together in element-wise operations.

In [2]:
import torch

## Broadcasting rules

To understand broadcasing consider its rules:

- **Rule 1 (dimensioning)**: If tensors have a different number of dimensions, the shape of the smaller tensor is padded with ones on the left side until both tensors have the same number of dimensions.

- **Rule 2 (compatability)**: Two dimensions are compatible when: They are equal, or one of them is 1

- **Rule 3 (broadcasting)**: If the dimensions are compatible, the tensors are broadcasted to the shape of the larger tensor. If not compatible, an error is raised.

Look at each rule separately with examples.

### Dimensioning

To apply element-wise operations, tensors must have the same number of dimensions. By default, if two tensors have different dimensionalities, the tensor with fewer dimensions will be expanded to match the higher-dimensional tensor. More formally, given two tensors $A$ and $B$ with dimensionalities $(d_1, d_2, ..., d_n)$ and $(d'_1, d'_2, ... , d'_m)$ respectively, where $m < n$, PyTorch will transform the dimensionality of $B$ to $(d''1, d''2, ..., d''{n-m}, d''{n-m+1}, ..., d''_n)$ where:

$$
d''_i = \begin{cases}
1, & \text{if } i \in \{1, 2, \ldots, n-m\}, \\
d'_j, & \text{if } i \in \{n-m+1, \ldots, n\} \text{ and } j = i - n + m,
\end{cases}
$$

Consider an example where we have tensors $A$ and $B$ with dimensionalities: $(1, 2, 3, 2, 3)$ and $(3, 2, 3)$ respectively.

By default, PyTorch will transform $B$ to have a dimensionality of $(1, 1, 3, 2, 3)$. It adds ones to the beginning of the dimensionality until it reaches the same length as $A$ (5 dimensions), and then uses the original dimensions of $B$.

When a tensor is transformed to match another tensor's dimensionality, it's effectively being wrapped in a series of additional dimensions, each of size 1. This creates a "shell" around the original tensor, expanding it to have the same number of dimensions as the target tensor. 

---

Let's consider a code example that demonstrates this dimensionality transformation. We'll add an element to a tensor and observe how PyTorch handles the broadcasting.

In [49]:
a = torch.arange(4).reshape(2,2)
a

tensor([[0, 1],
        [2, 3]])

As a second tensor, we'll use a one-dimensional tensor `b`:

In [51]:
b = torch.tensor([5, 6])
b

tensor([5, 6])

There could be two ways to add two elements to the tensor `a`. By rows and by columns the following cell shows both options.

In [47]:
print("Adding")
print(b[:, None])
a + b[:, None]

Adding
tensor([[1],
        [2]])


tensor([[1, 2],
        [4, 5]])

In [48]:
print("Adding", b[None])
a + b[None]

Adding tensor([[1, 2]])


tensor([[1, 3],
        [3, 5]])

Before we specify the exact method for adding the missing dimensionality, let's examine how PyTorch chooses the default approach. This will be demonstrated in the following cell.

In [44]:
a + b

tensor([[1, 3],
        [3, 5]])

By default, PyTorch added the extra dimension by rows. This behavior is because wrapping a one-dimensional tensor in an outer dimension results in a one-row matrix.  The two operands then share the column dimension, enabling element-wise operations.

## By row

Suppose you need to add a zero tensor to an array containing two numbers. So in the following cells the tensors `a` and `b` are created, which we will use for example:

In [3]:
a = torch.zeros((2, 3))
a

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [4]:
b = torch.tensor([2, 3])
b

tensor([2, 3])

So suppose we want to add the first element of `b` to the first row of `a` and the second element of `b` to the second row of `a`. Obviously just using operator `+` won't work.

In [6]:
a + b

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

The added tensor must be converted so that $n_1=m_1$. 

Where :
- $n_i$ - zeros tensor $i-th$ dimention size;
- $m_i$ - addimg tensor $i-th$ dimetion size.

So in the following example we will use `unsqueeze` for this purpose.

In [5]:
b_unsqueezed = b.unsqueeze(1)
b_unsqueezed

tensor([[2],
        [3]])

And finally result of sum. Essentially we just added the corresponding values of array `b` to each element in the row of array `a`:

In [7]:
a + b_unsqueezed

tensor([[2., 2., 2.],
        [3., 3., 3.]])

## By layer

This shows how to add a tensor of any dimension to another tensor along the selected axis. 

---

The following cell creates a tensor with the dimensionality `(3, 2, 3, 3)`. We'll interpret this as 3 cubes with 2 layers, 3 rows and 3 columns.

In [10]:
a = torch.zeros((3, 2, 3, 3))
a

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])

Suppose we need to add same number to the corresponding layer of each cube. Let it be 2 for first layer and 3 for second layer. So we have to create tensor with dimentions `(2,1,1)` - which you can understand as two elements in different layers.

In [19]:
b = torch.tensor([[[2]], [[3]]])
print(b)
print(b.shape)

tensor([[[2]],

        [[3]]])
torch.Size([2, 1, 1])


Now perform an addition operation on the corresponding layer of each cube.

In [21]:
a + b

tensor([[[[2., 2., 2.],
          [2., 2., 2.],
          [2., 2., 2.]],

         [[3., 3., 3.],
          [3., 3., 3.],
          [3., 3., 3.]]],


        [[[2., 2., 2.],
          [2., 2., 2.],
          [2., 2., 2.]],

         [[3., 3., 3.],
          [3., 3., 3.],
          [3., 3., 3.]]],


        [[[2., 2., 2.],
          [2., 2., 2.],
          [2., 2., 2.]],

         [[3., 3., 3.],
          [3., 3., 3.],
          [3., 3., 3.]]]])

## Practical example

Normalizing data can be achieved efficiently and easily using broadcasting techniques. Imagine we have four objects, each represented by a 3x3 matrix.  This gives us a three-dimensional tensor containing these matrices. However, each object has a different base value.  The following code snippet generates and prints such a tensor: 

In [146]:
value = torch.stack([
    torch.normal(mean=mean, std=3., size=(3, 3))
    for mean in [10, 100, 500, 1000]
])
value

tensor([[[  12.8460,   11.6038,    8.2267],
         [  13.3707,   13.0595,    9.3532],
         [  10.6257,   10.8800,    5.5372]],

        [[ 102.3054,   98.2987,   89.9491],
         [ 103.9013,   98.5896,   98.1971],
         [ 106.4493,   98.1544,   96.3743]],

        [[ 501.0322,  499.2918,  504.5105],
         [ 496.8454,  498.7290,  499.3281],
         [ 499.4231,  504.0831,  499.4671]],

        [[ 997.9377, 1001.2782, 1005.9980],
         [ 999.5461, 1001.3517,  997.5663],
         [1003.1572,  999.6476, 1001.6178]]])

Let's say we want to standardize all of these objects, meaning we want to bring them to zero mean and unit variance.

First, we need to calculate the mean and standard deviation for each object we're considering. 

In [147]:
mean = value.mean(axis=[1,2], keepdim=True)
print(mean)
std = value.std(axis=[1,2], keepdim=True)
print(std)

tensor([[[  10.6114]],

        [[  99.1355]],

        [[ 500.3011]],

        [[1000.9001]]])
tensor([[[2.5660]],

        [[4.7580]],

        [[2.5100]],

        [[2.6277]]])


The result is two vectors, each containing a value for every object in the input array. These vectors have the same dimensionality as the objects themselves - the `keepdim=True` argument ensures this. 

Now, by subtracting the means from the original arrays and dividing the result by the standard deviations, we've effectively scaled all the objects to the same scale. 

In [148]:
(value - mean)/std

tensor([[[ 0.8708,  0.3867, -0.9293],
         [ 1.0753,  0.9540, -0.4904],
         [ 0.0056,  0.1047, -1.9775]],

        [[ 0.6662, -0.1759, -1.9307],
         [ 1.0016, -0.1147, -0.1972],
         [ 1.5372, -0.2062, -0.5803]],

        [[ 0.2912, -0.4021,  1.6770],
         [-1.3768, -0.6263, -0.3877],
         [-0.3498,  1.5068, -0.3323]],

        [[-1.1274,  0.1439,  1.9401],
         [-0.5153,  0.1719, -1.2687],
         [ 0.8590, -0.4767,  0.2731]]])